In [ ]:
# Install necessary dependencies.
using Pkg
Pkg.activate(; temp=true)
Pkg.add([])

```julia
#| echo: false
#| output: false
using Pkg;
Pkg.instantiate();
```

This article provides an overview of the core functionality in Turing.jl, which are likely to be used across a wide range of models.

## Basics

### Introduction

A probabilistic program is a Julia function wrapped in a `@model` macro.
In this function, arbitrary Julia code can be used, but to ensure correctness of inference it should not have external effects or modify global state.

To specify distributions of random variables, Turing models use `~` notation: `x ~ distr` where `x` is an identifier.
This resembles the notation used in statistical models.
For example, the model:

$$\begin{align}
a &\sim \text{Normal}(0, 1) \\
x &\sim \text{Normal}(a, 1)
\end{align}$$

is written in Turing as:

```julia
using Turing

@model function mymodel()
    a ~ Normal(0, 1)
    x ~ Normal(a, 1)
end
```

### Tilde-statements

Indexing and field access is supported, so that `x[i] ~ distr` and `x.field ~ distr` are valid statements.
However, in these cases, `x` must be defined in the scope of the model function.
`distr` is typically either a distribution from Distributions.jl (see [this page]({{< meta usage-custom-distribution >}}) for implementing custom distributions), or another Turing model wrapped in `to_submodel()` (see [this page]({{< meta usage-submodels >}}) for submodels).

There are two classes of tilde-statements: _observe_ statements, where the left-hand side contains an observed value, and _assume_ statements, where the left-hand side is not observed.
These respectively correspond to likelihood and prior terms.

It is easier to start by explaining when a variable is treated as an observed value.
This can happen in one of two ways:

- The variable is passed as one of the arguments to the model function; or
- The value of the variable in the model is explicitly conditioned or fixed.

> Note that it is not enough for the variable to be defined in the current scope. For example, in
> 
> ```julia
> @model function mymodel(x)
>     y = x + 1
>     y ~ Normal(0, 1)
> end
> ```
> 
> `y` is not treated as an observed value.

In such a case, `x` is considered to be an observed value, assumed to have been drawn from the distribution `distr`.
The likelihood (if needed) is computed using `loglikelihood(distr, x)`.

On the other hand, if neither of the above are true, then this is treated as an assume-statement: inside the probabilistic program, this samples a new variable called `x`, distributed according to `distr`, and places it in the current scope.

### Simple Gaussian Demo

Below is a simple Gaussian demo illustrating the basic usage of Turing.jl.

```julia
# Import packages.
using Turing
using StatsPlots

# Define a simple Normal model with unknown mean and variance.
@model function gdemo(x, y)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    x ~ Normal(m, sqrt(s²))
    return y ~ Normal(m, sqrt(s²))
end
```

In Turing.jl, MCMC sampling is performed using the `sample()` function, which (at its most basic) takes a model, a sampler, and the number of samples to draw.

For this model, the prior expectation of `s²` is `mean(InverseGamma(2, 3)) = 3/(2 - 1) = 3`, and the prior expectation of `m` is 0.
We can check this using the `Prior` sampler:

```julia
#| output: false
setprogress!(false)
```

In [ ]:
p1 = sample(gdemo(missing, missing), Prior(), 100000)

To perform inference, we simply need to specify the sampling algorithm we want to use.

```julia
#  Run sampler, collect results.
c1 = sample(gdemo(1.5, 2), SMC(), 1000)
c2 = sample(gdemo(1.5, 2), PG(10), 1000)
c3 = sample(gdemo(1.5, 2), HMC(0.1, 5), 1000)
c4 = sample(gdemo(1.5, 2), Gibbs(:m => PG(10), :s² => HMC(0.1, 5)), 1000)
c5 = sample(gdemo(1.5, 2), HMCDA(0.15, 0.65), 1000)
c6 = sample(gdemo(1.5, 2), NUTS(0.65), 1000)
```

The arguments for each sampler are:

  - SMC: number of particles.
  - PG: number of particles, number of iterations.
  - HMC: leapfrog step size, leapfrog step numbers.
  - Gibbs: component sampler 1, component sampler 2, ...
  - HMCDA: total leapfrog length, target accept ratio.
  - NUTS: number of adaptation steps (optional), target accept ratio.

More information about each sampler can be found in [Turing.jl's API docs](https://turinglang.org/Turing.jl).

The `MCMCChains` module (which is re-exported by Turing) provides plotting tools for the `Chain` objects returned by a `sample` function.
See the [MCMCChains](https://github.com/TuringLang/MCMCChains.jl) repository for more information on the suite of tools available for diagnosing MCMC chains.

```julia
# Summarise results
describe(c3)

# Plot results
plot(c3)
savefig("gdemo-plot.png")
```

### Conditioning on data

Using this syntax, a probabilistic model is defined in Turing.
The model function generated by Turing can then be used to condition the model on data.
Subsequently, the `sample` function can be used to generate samples from the posterior distribution.

In the following example, the defined model is conditioned to the data (`arg_1 = 1`, `arg_2 = 2`) by passing the arguments `1` and `2` to the model function.

```julia
@model function model_name(arg_1, arg_2)
    arg_1 ~ ...
    arg_2 ~ ...
end
```

The conditioned model can then be passed onto the sample function to run posterior inference.

```julia
model = model_name(1, 2)
chn = sample(model, HMC(0.5, 20), 1000) # Sample with HMC.
```

Alternatively, one can also use the conditioning operator `|` to condition the model on data.
In this case, the model does not need to be defined with `arg_1` and `arg_2` as parameters.

```julia
@model function model_name()
    arg_1 ~ ...
    arg_2 ~ ...
end

# Condition the model on data.
model = model_name() | (arg_1 = 1, arg_2 = 2) 
```

### Analysing MCMC chains

The returned chain contains samples of the variables in the model.

```julia
var_1 = mean(chn[:var_1]) # Taking the mean of a variable named var_1.
```

The key (`:var_1`) can either be a `Symbol` or a `String`.
For example, to fetch `x[1]`, one can use `chn[Symbol("x[1]")]` or `chn["x[1]"]`.
If you want to retrieve all parameters associated with a specific symbol, you can use `group`.
As an example, if you have the parameters `"x[1]"`, `"x[2]"`, and `"x[3]"`, calling `group(chn, :x)` or `group(chn, "x")` will return a new chain with only `"x[1]"`, `"x[2]"`, and `"x[3]"`.

### Tilde-statement ordering

Turing does not have a declarative form.
Thus, the ordering of tilde-statements in a Turing model is important: random variables cannot be used until they have been first declared in a tilde-statement.
For example, the following example works:

```julia
# Define a simple Normal model with unknown mean and variance.
@model function model_function(y)
    s ~ Poisson(1)
    y ~ Normal(s, 1)
    return y
end

sample(model_function(10), SMC(), 100)
```

But if we switch the `s ~ Poisson(1)` and `y ~ Normal(s, 1)` lines, the model will no longer sample correctly:

```julia
# Define a simple Normal model with unknown mean and variance.
@model function model_function(y)
    y ~ Normal(s, 1)
    s ~ Poisson(1)
    return y
end

sample(model_function(10), SMC(), 100)
```

### Sampling Multiple Chains

Turing supports distributed and threaded parallel sampling.
To do so, call `sample(model, sampler, parallel_type, n, n_chains)`, where `parallel_type` can be either `MCMCThreads()` or `MCMCDistributed()` for thread and parallel sampling, respectively.

Having multiple chains in the same object is valuable for evaluating convergence.
Some diagnostic functions like `gelmandiag` require multiple chains.

If you want to sample multiple chains without using parallelism, you can use `MCMCSerial()`:

```julia
# Sample 3 chains in a serial fashion.
chains = sample(model, sampler, MCMCSerial(), 1000, 3)
```

The `chains` variable now contains a `Chains` object which can be indexed by chain. To pull out the first chain from the `chains` object, use `chains[:,:,1]`. The method is the same if you use either of the below parallel sampling methods.

#### Multithreaded sampling

If you wish to perform multithreaded sampling, you can call `sample` with the following signature:

```julia
# Sample four chains using multiple threads, each with 1000 samples.
sample(model, sampler, MCMCThreads(), 1000, 4)
```

Be aware that Turing cannot add threads for you -- you must have started your Julia instance with multiple threads to experience any kind of parallelism.
See the [Julia documentation](https://docs.julialang.org/en/v1/manual/parallel-computing/#man-multithreading-1) for details on how to achieve this.

#### Distributed sampling

To perform distributed sampling (using multiple processes), you must first import `Distributed`.

Process parallel sampling can be done like so:

```julia
#| eval: false
# Load Distributed to add processes and the @everywhere macro.
using Distributed

# Load Turing.
using Turing

# Add four processes to use for sampling.
addprocs(4; exeflags="--project=$(Base.active_project())")

# Initialise everything on all the processes.
# Note: Make sure to do this after you've already loaded Turing,
#       so each process does not have to precompile.
#       Parallel sampling may fail silently if you do not do this.
@everywhere using Turing

# Define a model on all processes.
@everywhere @model function gdemo(x)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))

    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s²))
    end
end

# Declare the model instance everywhere.
@everywhere model = gdemo([1.5, 2.0])

# Sample four chains using multiple processes, each with 1000 samples.
sample(model, NUTS(), MCMCDistributed(), 1000, 4)
```

### Sampling from an Unconditional Distribution (The Prior)

Turing allows you to sample from a declared model's prior. If you wish to draw a chain from the prior to inspect your prior distributions, you can run

```julia
#| eval: false
chain = sample(model, Prior(), n_samples)
```

You can also run your model (as if it were a function) from the prior distribution, by calling the model without specifying inputs or a sampler.
In the below example, we specify a `gdemo` model which returns two variables, `x` and `y`.
Here, including the `return` statement is necessary to retrieve the sampled `x` and `y` values.

In [ ]:
@model function gdemo(x, y)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    x ~ Normal(m, sqrt(s²))
    y ~ Normal(m, sqrt(s²))
    return x, y
end

To produce a sample from the prior distribution, we instantiate the model with `missing` inputs:

```julia
# Samples from p(x,y)
g_prior_sample = gdemo(missing, missing)
g_prior_sample()
```

### Sampling from a Conditional Distribution (The Posterior)

#### Treating observations as random variables

Inputs to the model that have a value `missing` are treated as parameters, aka random variables, to be estimated/sampled. This can be useful if you want to simulate draws for that parameter, or if you are sampling from a conditional distribution. Turing supports the following syntax:

```julia
@model function gdemo(x, ::Type{T}=Float64) where {T}
    if x === missing
        # Initialise `x` if missing
        x = Vector{T}(undef, 2)
    end
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s²))
    end
end

# Construct a model with x = missing
model = gdemo(missing)
c = sample(model, HMC(0.05, 20), 500)
```

Note the need to initialise `x` when missing since we are iterating over its elements later in the model.
The generated values for `x` can be extracted from the `Chains` object using `c[:x]`.

Turing also supports mixed `missing` and non-`missing` values in `x`, where the missing ones will be treated as random variables to be sampled while the others get treated as observations.
For example:

```julia
@model function gdemo(x)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s²))
    end
end

# x[1] is a parameter, but x[2] is an observation
model = gdemo([missing, 2.4])
c = sample(model, HMC(0.01, 5), 500)
```

#### Default Values

Arguments to Turing models can have default values much like how default values work in normal Julia functions.
For instance, the following will assign `missing` to `x` and treat it as a random variable.
If the default value is not `missing`, `x` will be assigned that value and will be treated as an observation instead.

```julia
using Turing

@model function generative(x=missing, ::Type{T}=Float64) where {T<:Real}
    if x === missing
        # Initialise x when missing
        x = Vector{T}(undef, 10)
    end
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    for i in 1:length(x)
        x[i] ~ Normal(m, sqrt(s²))
    end
    return s², m
end

m = generative()
chain = sample(m, HMC(0.01, 5), 1000)
```

#### Access Values inside Chain

You can access the values inside a chain in several ways:

 1. Turn them into a `DataFrame` object
 2. Use their raw `AxisArray` form
 3. Create a three-dimensional `Array` object

For example, let `c` be a `Chain`:

 1. `DataFrame(c)` converts `c` to a `DataFrame`,
 2. `c.value` retrieves the values inside `c` as an `AxisArray`, and
 3. `c.value.data` retrieves the values inside `c` as a 3D `Array`.

#### Variable Types and Type Parameters

The element type of a vector (or matrix) of random variables should match the `eltype` of its prior distribution, i.e., `<: Integer` for discrete distributions and `<: AbstractFloat` for continuous distributions.

Some automatic differentiation backends (used in conjunction with Hamiltonian samplers such as `HMC` or `NUTS`) further require that the vector's element type needs to either be:

1. `Real` to enable auto-differentiation through the model which uses special number types that are sub-types of `Real`, or

2. Some type parameter `T` defined in the model header using the type parameter syntax, e.g. `function gdemo(x, ::Type{T} = Float64) where {T}`.

Similarly, when using a particle sampler, the Julia variable used should either be:

1. An `Array`, or

2. An instance of some type parameter `T` defined in the model header using the type parameter syntax, e.g. `function gdemo(x, ::Type{T} = Vector{Float64}) where {T}`.

### Querying Probabilities from Model or Chain

Turing offers three functions: [`loglikelihood`](https://turinglang.org/DynamicPPL.jl/dev/api/#StatsAPI.loglikelihood), [`logprior`](https://turinglang.org/DynamicPPL.jl/dev/api/#DynamicPPL.logprior), and [`logjoint`](https://turinglang.org/DynamicPPL.jl/dev/api/#DynamicPPL.logjoint) to query the log-likelihood, log-prior, and log-joint probabilities of a model, respectively.

Let's look at a simple model called `gdemo`:

In [ ]:
@model function gdemo0()
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    return x ~ Normal(m, sqrt(s))
end

If we observe x to be 1.0, we can condition the model on this datum using the [`condition`](https://turinglang.org/DynamicPPL.jl/dev/api/#AbstractPPL.condition) syntax:

In [ ]:
model = gdemo0() | (x=1.0,)

Now, let's compute the log-likelihood of the observation given specific values of the model parameters, `s` and `m`:

In [ ]:
loglikelihood(model, (s=1.0, m=1.0))

We can easily verify that value in this case:

In [ ]:
logpdf(Normal(1.0, 1.0), 1.0)

We can also compute the log-prior probability of the model for the same values of s and m:

In [ ]:
logprior(model, (s=1.0, m=1.0))

In [ ]:
logpdf(InverseGamma(2, 3), 1.0) + logpdf(Normal(0, sqrt(1.0)), 1.0)

Finally, we can compute the log-joint probability of the model parameters and data:

In [ ]:
logjoint(model, (s=1.0, m=1.0))

In [ ]:
logpdf(Normal(1.0, 1.0), 1.0) +
logpdf(InverseGamma(2, 3), 1.0) +
logpdf(Normal(0, sqrt(1.0)), 1.0)

Querying with `Chains` object is easy as well:

In [ ]:
chn = sample(model, Prior(), 10)

In [ ]:
loglikelihood(model, chn)

### Maximum likelihood and maximum a posteriori estimates

Turing also has functions for estimating the maximum a posteriori and maximum likelihood parameters of a model. This can be done with

In [ ]:
mle_estimate = maximum_likelihood(model)
map_estimate = maximum_a_posteriori(model)

For more details see the [mode estimation page]({{<meta usage-mode-estimation>}}).

## Beyond the Basics

### Compositional Sampling Using Gibbs

Turing.jl provides a Gibbs interface to combine different samplers. For example, one can combine an `HMC` sampler with a `PG` sampler to run inference for different parameters in a single model as below.

In [ ]:
@model function simple_choice(xs)
    p ~ Beta(2, 2)
    z ~ Bernoulli(p)
    for i in 1:length(xs)
        if z == 1
            xs[i] ~ Normal(0, 1)
        else
            xs[i] ~ Normal(2, 1)
        end
    end
end

simple_choice_f = simple_choice([1.5, 2.0, 0.3])

chn = sample(simple_choice_f, Gibbs(:p => HMC(0.2, 3), :z => PG(20)), 1000)

The `Gibbs` sampler can be used to specify unique automatic differentiation backends for different variable spaces. Please see the [Automatic Differentiation]({{<meta usage-automatic-differentiation>}}) article for more.

For more details of compositional sampling in Turing.jl, please check the corresponding [paper](https://proceedings.mlr.press/v84/ge18b.html).

### Working with `filldist` and `arraydist`

Turing provides `filldist(dist::Distribution, n::Int)` and `arraydist(dists::AbstractVector{<:Distribution})` as a simplified interface to construct product distributions, e.g., to model a set of variables that share the same structure but vary by group.

#### Constructing product distributions with filldist

The function `filldist` provides a general interface to construct product distributions over distributions of the same type and parameterisation.
Note that, in contrast to the product distribution interface provided by Distributions.jl (`Product`), `filldist` supports product distributions over univariate or multivariate distributions.

Example usage:

```julia
@model function demo(x, g)
    k = length(unique(g))
    a ~ filldist(Exponential(), k) # = Product(fill(Exponential(), k))
    mu = a[g]
    for i in eachindex(x)
        x[i] ~ Normal(mu[i])
    end
    return mu
end
```

#### Constructing product distributions with `arraydist`

The function `arraydist` provides a general interface to construct product distributions over distributions of varying type and parameterisation.
Note that in contrast to the product distribution interface provided by Distributions.jl (`Product`), `arraydist` supports product distributions over univariate or multivariate distributions.

Example usage:

In [ ]:
@model function demo(x, g)
    k = length(unique(g))
    a ~ arraydist([Exponential(i) for i in 1:k])
    mu = a[g]
    for i in eachindex(x)
        x[i] ~ Normal(mu[i])
    end
    return mu
end

### Working with MCMCChains.jl

Turing.jl wraps its samples using `MCMCChains.Chain` so that all the functions working for `MCMCChains.Chain` can be re-used in Turing.jl. Two typical functions are `MCMCChains.describe` and `MCMCChains.plot`, which can be used as follows for an obtained chain `chn`. For more information on `MCMCChains`, please see the [GitHub repository](https://github.com/TuringLang/MCMCChains.jl).

```julia
describe(chn) # Lists statistics of the samples.
plot(chn) # Plots statistics of the samples.
```

There are numerous functions in addition to `describe` and `plot` in the `MCMCChains` package, such as those used in convergence diagnostics. For more information on the package, please see the [GitHub repository](https://github.com/TuringLang/MCMCChains.jl).

### Changing Default Settings

Some of Turing.jl's default settings can be changed for better usage.

#### AD Backend

Turing is thoroughly tested with three automatic differentiation (AD) backend packages.
The default AD backend is [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl), which uses forward-mode AD.
Two reverse-mode AD backends are also supported, namely [Mooncake](https://github.com/compintell/Mooncake.jl) and [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl).
`Mooncake` and `ReverseDiff` also require the user to explicitly load them using `import Mooncake` or `import ReverseDiff` next to `using Turing`.

For more information on Turing's automatic differentiation backend, please see the [Automatic Differentiation]({{<meta usage-automatic-differentiation>}}) article as well as the [ADTests website](https://turinglang.org/ADTests/), where a number of AD backends (not just those above) are tested against Turing.jl.

#### Progress Logging

`Turing.jl` uses ProgressLogging.jl to log the sampling progress. Progress
logging is enabled as default but might slow down inference. It can be turned on
or off by setting the keyword argument `progress` of `sample` to `true` or `false`.
Moreover, you can enable or disable progress logging globally by calling `setprogress!(true)` or `setprogress!(false)`, respectively.

Turing uses heuristics to select an appropriate visualisation backend. If you
use Jupyter notebooks, the default backend is
[ConsoleProgressMonitor.jl](https://github.com/tkf/ConsoleProgressMonitor.jl).
In all other cases, progress logs are displayed with
[TerminalLoggers.jl](https://github.com/c42f/TerminalLoggers.jl). Alternatively,
if you provide a custom visualisation backend, Turing uses it instead of the
default backend.